# Description

The notebook demonstrates how open-source solvers solve the DaoSwap problem.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
import logging
from typing import List, Tuple

import numpy as np
import pandas as pd

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import defi.dao_cross.optimize as ddacropt
import defi.dao_cross.order as ddacrord


In [3]:
try:
    import pulp
except ImportError:
    !sudo /bin/bash -c "(source /venv/bin/activate; pip install pulp)"
    import pulp

In [4]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-f728515a-aaa0-4817-8a8d-23a321969184.json'
-----------------------------------------------------------------------------
This code is not in sync with the container:
code_version='1.4.1' != container_version='1.4.0'
-----------------------------------------------------------------------------
You need to:
- merge origin/master into your branch with `invoke git_merge_master`
- pull the latest container with `invoke docker_pull`
INFO  # Git
  branch_name='CmTask80_Implement_DaoSwap_optimization_problem_1'
  hash='af9f29ab9'
  # Last commits:
    * af9f29ab9 paul     Checkpoint                                                        (26 minutes ago) Tue Apr 18 19:57:14 2023  (HEAD -> CmTask80_Implement_DaoSwap_optimization_problem_1)
    * 098651322 Paul     Cm task4011 write dao swap white paper 11 (#147)                  (   2 hours ago) Tue Apr 18 18:14:10 2023  (origin/mast

# Test orders

In [5]:
def _generate_test_orders(
    actions: List[str],
    quantities: List[float],
    base_tokens: List[str],
    limit_prices: List[float],
    quote_tokens: List[str],
) -> List[ddacrord.Order]:
    """
    Create N `Order` instances using the inputs.

    See `Order` for params description.
    """
    # Use dummy values as the params are not relevant for the
    # optimization problem.
    timestamp = np.nan
    deposit_address = 1
    wallet_address = 1
    orders: List[ddacrord.Order] = []
    # TODO(Grisha): check that all lists are of the same length.
    for i in range(len(base_tokens)):
        order_i = ddacrord.Order(
            timestamp,
            actions[i],
            quantities[i],
            base_tokens[i],
            limit_prices[i],
            quote_tokens[i],
            deposit_address,
            wallet_address,
        )
        orders.append(order_i)
    return orders

In [6]:
_actions = ["buy", "buy", "sell", "sell", "buy", "buy", "sell", "sell"]
_quantities = [4, 2, 5, 3, 6, 2, 9, 1]
_base_tokens = ["BTC", "BTC", "BTC", "BTC", "ETH", "ETH", "ETH", "ETH"]
_quote_tokens = ["ETH", "ETH", "ETH", "ETH", "BTC", "BTC", "BTC", "BTC"]
_limit_prices = [3, 3.5, 1.5, 1.9, 0.6, 2, 0.1, 0.25]

def test1() -> None:
    """
    The limit price condition is True for all orders.
    """
    # Get inputs.
    prices = {"BTC": 3, "ETH": 6}
    limit_prices = [3, 3.5, 1.5, 1.9, 0.6, 2, 0.1, 0.25]
    test_orders = _generate_test_orders(
        _actions,
        _quantities,
        _base_tokens,
        _limit_prices,
        _quote_tokens,
    )
    return test_orders

In [57]:
prices = {"BTC": 3, "ETH": 6}

In [59]:
orders = test1()

In [60]:
ddacrord.convert_orders_to_dataframe(orders)

,timestamp,action,quantity,base_token,limit_price,quote_token,deposit_address
0,2023-04-19 13:47:18.122212+00:00,buy,4,BTC,3.0,ETH,1
1,2023-04-19 13:47:18.122370+00:00,buy,2,BTC,3.5,ETH,1
2,2023-04-19 13:47:18.122419+00:00,sell,5,BTC,1.5,ETH,1
3,2023-04-19 13:47:18.122460+00:00,sell,3,BTC,1.9,ETH,1
4,2023-04-19 13:47:18.122499+00:00,buy,6,ETH,0.6,BTC,1
5,2023-04-19 13:47:18.122537+00:00,buy,2,ETH,2.0,BTC,1
6,2023-04-19 13:47:18.122575+00:00,sell,9,ETH,0.1,BTC,1
7,2023-04-19 13:47:18.122611+00:00,sell,1,ETH,0.25,BTC,1


## Run DaoCross

In [62]:
daocross_results = ddacropt.run_daocross_solver(orders, prices)
display(daocross_results)

/venv/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


,timestamp,action,quantity,base_token,limit_price,quote_token,deposit_address,q_pi_star,q_tau_star,effective_price
0,2023-04-19 13:47:18.122212+00:00,buy,4,BTC,3.0,ETH,1,4.0,8.0,2.0
1,2023-04-19 13:47:18.122370+00:00,buy,2,BTC,3.5,ETH,1,2.0,4.0,2.0
2,2023-04-19 13:47:18.122419+00:00,sell,5,BTC,1.5,ETH,1,5.0,10.0,2.0
3,2023-04-19 13:47:18.122460+00:00,sell,3,BTC,1.9,ETH,1,2.0,4.0,2.0
4,2023-04-19 13:47:18.122499+00:00,buy,6,ETH,0.6,BTC,1,6.0,3.0,0.5
5,2023-04-19 13:47:18.122537+00:00,buy,2,ETH,2.0,BTC,1,2.0,1.0,0.5
6,2023-04-19 13:47:18.122575+00:00,sell,9,ETH,0.1,BTC,1,9.0,4.5,0.5
7,2023-04-19 13:47:18.122611+00:00,sell,1,ETH,0.25,BTC,1,1.0,0.5,0.5


In [ ]:
## Run DaoSwap

In [63]:
daoswap_results = ddacropt.run_daoswap_solver(orders)
display(daoswap_results)

/venv/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


,timestamp,action,quantity,base_token,limit_price,quote_token,deposit_address,q_pi_star,q_tau_star,effective_price
0,2023-04-19 13:47:18.122212+00:00,buy,4,BTC,3.0,ETH,1,4.0,12.0,3.0
1,2023-04-19 13:47:18.122370+00:00,buy,2,BTC,3.5,ETH,1,2.0,7.0,3.5
2,2023-04-19 13:47:18.122419+00:00,sell,5,BTC,1.5,ETH,1,5.0,7.5,1.5
3,2023-04-19 13:47:18.122460+00:00,sell,3,BTC,1.9,ETH,1,3.0,13.5,4.5
4,2023-04-19 13:47:18.122499+00:00,buy,6,ETH,0.6,BTC,1,6.0,3.6,0.6
5,2023-04-19 13:47:18.122537+00:00,buy,2,ETH,2.0,BTC,1,2.0,4.0,2.0
6,2023-04-19 13:47:18.122575+00:00,sell,9,ETH,0.1,BTC,1,9.0,0.9,0.1
7,2023-04-19 13:47:18.122611+00:00,sell,1,ETH,0.25,BTC,1,1.0,8.7,8.7


In [49]:
result_df

,timestamp,action,quantity,base_token,limit_price,quote_token,deposit_address,q_pi_star,q_tau_star,effective_price
0,2023-04-18 20:23:28.635157+00:00,buy,4,BTC,3.0,ETH,1,4.0,12.0,3.0
1,2023-04-18 20:23:28.636897+00:00,buy,2,BTC,3.5,ETH,1,2.0,7.0,3.5
2,2023-04-18 20:23:28.636998+00:00,sell,5,BTC,1.5,ETH,1,5.0,7.5,1.5
3,2023-04-18 20:23:28.637044+00:00,sell,3,BTC,1.9,ETH,1,3.0,13.5,4.5
4,2023-04-18 20:23:28.637086+00:00,buy,6,ETH,0.6,BTC,1,6.0,3.6,0.6
5,2023-04-18 20:23:28.637127+00:00,buy,2,ETH,2.0,BTC,1,2.0,4.0,2.0
6,2023-04-18 20:23:28.637165+00:00,sell,9,ETH,0.1,BTC,1,9.0,0.9,0.1
7,2023-04-18 20:23:28.637211+00:00,sell,1,ETH,0.25,BTC,1,1.0,8.7,8.7


In [16]:
effective_price = [result["q_tau_star"][i] / result["q_pi_star"][i] for i in range(len(orders))]

In [13]:
effective_price

[3.0, 3.5, 1.5, 4.5, 0.6, 2.0, 0.1, 8.7]